#### What-if Query Execution Cost estimation for Hypothetical indexes

In [15]:
%load_ext autoreload
%autoreload 2

from ssb_qgen_class import *
from pg_utils import *


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
# create a query generator object
qg = QGEN()

In [13]:
query = qg.generate_query(1)
print(query)

template id: 1, query: 
                SELECT SUM(lo_extendedprice * lo_discount) AS revenue
                FROM lineorder, dwdate
                WHERE lo_orderdate = d_datekey
                AND d_year = 1993
                AND lo_discount BETWEEN 6 AND 8 
                AND lo_quantity < 14;
            , payload: {'lineorder': ['lo_extendedprice', 'lo_discount']}, predicates: {'lineorder': ['lo_orderdate', 'lo_discount', 'lo_quantity'], 'dwdate': ['d_datekey', 'd_year']}, order by: {}, group by: {}


In [22]:
candidate_indexes = extract_query_indexes(query)

for index in candidate_indexes:
    print(index)


Index name: IX_lineorder_lo_orderdate, Key cols: ('lo_orderdate',), Include cols: ()
Index name: IX_lineorder_lo_discount, Key cols: ('lo_discount',), Include cols: ()
Index name: IX_lineorder_lo_quantity, Key cols: ('lo_quantity',), Include cols: ()
Index name: IX_lineorder_lo_orderdate_lo_discount, Key cols: ('lo_orderdate', 'lo_discount'), Include cols: ()
Index name: IX_lineorder_lo_orderdate_lo_quantity, Key cols: ('lo_orderdate', 'lo_quantity'), Include cols: ()
Index name: IX_lineorder_lo_discount_lo_quantity, Key cols: ('lo_discount', 'lo_quantity'), Include cols: ()
Index name: IX_lineorder_lo_orderdate_lo_discount_lo_quantity, Key cols: ('lo_orderdate', 'lo_discount', 'lo_quantity'), Include cols: ()
Index name: IX_dwdate_d_datekey, Key cols: ('d_datekey',), Include cols: ()
Index name: IX_dwdate_d_year, Key cols: ('d_year',), Include cols: ()
Index name: IX_dwdate_d_datekey_d_year, Key cols: ('d_datekey', 'd_year'), Include cols: ()


In [27]:
conn = create_connection()
cost = get_query_cost_estimate(conn, query.query_string, show_plan=True)
print(f"Estimated cost: {cost}")

close_connection(conn)

[
  {
    "Plan": {
      "Node Type": "Aggregate",
      "Strategy": "Plain",
      "Partial Mode": "Finalize",
      "Parallel Aware": false,
      "Async Capable": false,
      "Startup Cost": 1427748.32,
      "Total Cost": 1427748.33,
      "Plan Rows": 1,
      "Plan Width": 32,
      "Plans": [
        {
          "Node Type": "Gather",
          "Parent Relationship": "Outer",
          "Parallel Aware": false,
          "Async Capable": false,
          "Startup Cost": 1427748.1,
          "Total Cost": 1427748.31,
          "Plan Rows": 2,
          "Plan Width": 32,
          "Workers Planned": 2,
          "Single Copy": false,
          "Plans": [
            {
              "Node Type": "Aggregate",
              "Strategy": "Plain",
              "Partial Mode": "Partial",
              "Parent Relationship": "Outer",
              "Parallel Aware": false,
              "Async Capable": false,
              "Startup Cost": 1426748.1,
              "Total Cost": 1426748.1